# שילוב Semantic Kernel עם שרת OpenBnB MCP

מחברת זו מדגימה כיצד להשתמש ב-Semantic Kernel עם שרת OpenBnB MCP בפועל כדי לחפש מקומות אירוח אמיתיים של Airbnb באמצעות MCPStdioPlugin. לגישה ל-LLM, נעשה שימוש ב-Azure AI Foundry. כדי להגדיר את משתני הסביבה שלך, תוכל לעקוב אחר [שיעור ההגדרה](/00-course-setup/README.md)


## ייבוא החבילות הנדרשות


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## יצירת חיבור תוסף MCP

נחבר ל-[שרת OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) באמצעות MCPStdioPlugin. שרת זה מספק יכולת חיפוש של Airbnb דרך החבילה @openbnb/mcp-server-airbnb.


## יצירת הלקוח

בדוגמה זו, נשתמש ב-Azure AI Foundry לצורך גישה ל-LLM. ודא שהמשתנים הסביבתיים שלך מוגדרים כראוי.


## הגדרת סביבת עבודה

הגדר את ההגדרות של Azure OpenAI. ודא שהגדרת את משתני הסביבה הבאים:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## הבנת האינטגרציה של OpenBnB MCP

מחברת זו מתחברת ל**שרת OpenBnB MCP האמיתי** שמספק פונקציונליות חיפוש אמיתית של Airbnb.

### איך זה עובד:

1. **MCPStdioPlugin**: משתמש בתקשורת קלט/פלט סטנדרטית עם שרת MCP  
2. **חבילת NPM אמיתית**: מוריד ומפעיל `@openbnb/mcp-server-airbnb` באמצעות npx  
3. **נתונים חיים**: מחזיר נתוני נכסים אמיתיים של Airbnb מתוך ה-APIs שלהם  
4. **גילוי פונקציות**: הסוכן מגלה באופן אוטומטי פונקציות זמינות משרת MCP  

### פונקציות זמינות:

שרת OpenBnB MCP בדרך כלל מספק:  
- **search_listings** - חיפוש נכסי Airbnb לפי מיקום וקריטריונים  
- **get_listing_details** - קבלת מידע מפורט על נכסים ספציפיים  
- **check_availability** - בדיקת זמינות לתאריכים מסוימים  
- **get_reviews** - שליפת ביקורות על נכסים  
- **get_host_info** - קבלת מידע על מארחי נכסים  

### דרישות מקדימות:

- **Node.js** מותקן במערכת שלך  
- **חיבור לאינטרנט** להורדת חבילת שרת MCP  
- **NPX** זמין (מגיע עם Node.js)  

### בדיקת החיבור:

ניתן לבדוק את שרת MCP באופן ידני על ידי הרצה של:  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

פעולה זו תוריד ותפעיל את שרת OpenBnB MCP, אליו Semantic Kernel מתחבר כדי לקבל נתוני Airbnb אמיתיים.  


## הפעלת הסוכן עם שרת OpenBnB MCP

כעת נפעיל את סוכן הבינה המלאכותית שמתחבר לשרת OpenBnB MCP כדי לחפש מקומות אירוח אמיתיים של Airbnb בסטוקהולם עבור 2 מבוגרים וילד אחד. אתם מוזמנים לשנות את הרשימה `user_inputs` כדי להתאים את קריטריוני החיפוש.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

סיכום  
ברכות! הצלחתם לבנות סוכן AI שמשתלב עם חיפוש מקומות אירוח בעולם האמיתי באמצעות פרוטוקול הקשר של מודלים (MCP):

טכנולוגיות בשימוש:  
Semantic Kernel - לבניית סוכנים חכמים עם Azure OpenAI  
Azure AI Foundry - ליכולות מודלים גדולים ושיחות  
MCP (Model Context Protocol) - לאינטגרציה סטנדרטית של כלים  
OpenBnB MCP Server - לפונקציונליות חיפוש אמיתי ב-Airbnb  
Node.js/NPX - להפעלת שרת MCP חיצוני  

מה שלמדתם:  
אינטגרציה עם MCP: חיבור סוכני Semantic Kernel לשרתי MCP חיצוניים  
גישה לנתונים בזמן אמת: חיפוש נכסי Airbnb אמיתיים דרך APIs חיים  
תקשורת פרוטוקול: שימוש בתקשורת stdio בין הסוכן לשרת MCP  
גילוי פונקציות: גילוי אוטומטי של פונקציות זמינות משרתי MCP  
תגובות זורמות: לכידת ורישום קריאות פונקציות בזמן אמת  
עיבוד HTML: עיצוב תגובות הסוכן עם טבלאות מעוצבות ותצוגות אינטראקטיביות  

הצעדים הבאים:  
אינטגרציה של שרתי MCP נוספים (מזג אוויר, טיסות, מסעדות)  
בניית מערכת רב-סוכנים שמשלבת פרוטוקולי MCP ו-A2A  
יצירת שרתי MCP מותאמים אישית למקורות הנתונים שלכם  
יישום זיכרון שיחה מתמשך בין סשנים  
פריסת הסוכן ל-Azure Functions עם תזמור שרת MCP  
הוספת יכולות אימות משתמש והזמנה  

יתרונות מרכזיים של ארכיטקטורת MCP:  
סטנדרטיזציה: פרוטוקול אוניברסלי לחיבור סוכני AI לכלים חיצוניים  
נתונים בזמן אמת: גישה למידע חי ומעודכן משירותים שונים  
הרחבה: אינטגרציה קלה של מקורות נתונים וכלים חדשים  
יכולת פעולה הדדית: עובד עם מסגרות AI ופלטפורמות סוכנים שונות  
הפרדת תחומים: הבחנה ברורה בין לוגיקת AI לגישה לנתונים חיצוניים  



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
